
#  Programming Languages (5) --- Generic functions and types


Enter your name and student ID.

 * Name:
 * Student ID:



# 1. Choose your language
From this week and on, choose the language you are assigned to

Declare your choice in the following cell (leave the one you choose).  In this notebook, I will work on

BEGIN SOLUTION
END SOLUTION
Go
Julia
OCaml
Rust

# 2. Roadmap
* In this notebook, you are going to learn _generic_ functions and types

* A _generic function_ (_type_) refers a function (_type_) that can be instantiated with many types
* A generic function is very useful because it can apply to many different types with a single definition

* Below, you are going to write a function that optimizes (minimizes) a function (_objective function_) given as a parameter, over a domain, which is also given as a parameter. i.e.,  ${\tt minimize}(f, D)$
* Wouldn't it be nice if we can implement it for functions that take, say, a floating point number, an integer, a pair of floating point numbers, etc., with a single definition?
* We can/will do this as follows
1. write a domain class/object that generates elements from the domain, one at a time
1. write the function that repeatedly draws an element $x$ from the domain, evaluates the objective function at $x$, and records the minimum value encountered
 

# 3. A warm up
## <font color="green"> Problem 1 :  A function that takes a function </font>
* Write a function $f$ that takes a float-taking function $g$ and apply $g$ to 1.0 in your language
* That is, $f(g) = g(1.0)$
* Master the syntax representing a function type, though in some languages you don't have to write it up yourself


In [ ]:
BEGIN SOLUTION
END SOLUTION


In [ ]:
let f g = g 1.0;;

In [ ]:
f(g) = g(1.0)

In [ ]:
func f(g func (float64) float64) float64 { return g(1.0); }

In [ ]:
fn f(g : fn (f64) -> f64) -> f64 { g(1.0) }

# 4. A domain class for a floating point number
* Our ultimate goal is to write a _generic_ minimizer, but to make matters easier, let's first define a data structure that generates floating point numbers

## <font color="green"> Problem 2 :  A domain class for a floating point number</font>
Define a data structure `float_random_generator` (change the name according to case conventions/requirements of your language), which
 * takes $a$, $b$ and $n$ upon creation
 * has `next()` method that returns a random floating point number in the interval $[a, b]$ up to $n$ times, after which it returns a value that indicates no more values

Note: `next()` may return either a floating point number of a value that indicates "end of sequence (no more value)". the most natural way to do it depends on your language.  Here are some keywords you want to search for

 |      |                      |
 |------|----------------------|
 |Go    |multiple return values|
 |Julia |nothing               |
 |OCaml |option                |
 |Rust  |Option                |

* Learn how to generate a random number in your language from the authoritive source
* I encourage you to Google by yourself, but just in case you had difficult time finding it

 |                                                         |
 |---------------------------------------------------------|
 |[Go](https://pkg.go.dev/math/rand)                       |
 |[Julia](https://docs.julialang.org/en/v1/stdlib/Random/) |
 |[OCaml](https://ocaml.org/manual/5.1/api/Random.html)    |
 |[Rust](https://docs.rs/rand/latest/rand/)                |


In [ ]:
BEGIN SOLUTION
END SOLUTION


In [ ]:
class random_float_generator a b n = object
  val mutable i = 0
  method next =
    if i < n then
      let x = a +. (b -. a) *. (Random.float 1.0) in
      let _ = i <- i + 1 in
      Some(x)
    else
      None
end
;;

In [ ]:
mutable struct random_float_generator
    a :: Float64
    b :: Float64
    n :: Int64
    i :: Int64
end

function random_float_generator(a, b, n)
    random_float_generator(a, b, n, 0)
end

function next(rfg :: random_float_generator)
    if rfg.i < rfg.n 
        x = rfg.a + (rfg.b - rfg.a) * rand(Float64)
        rfg.i += 1
        x
    else
        nothing
    end
end

In [ ]:
import "math/rand"
type RandomFloatGenerator struct {
        a, b float64
        n int
        i int
        rng * rand.Rand
}

func mk_random_float_generator(a float64, b float64, n int) * RandomFloatGenerator {
        rng := rand.New(rand.NewSource(123456));
        return &RandomFloatGenerator{a, b, n, 0, rng};
}

func (rfg * RandomFloatGenerator) next() (float64, bool) {
        if rfg.i < rfg.n {
                x := rfg.a + (rfg.b - rfg.a) * rfg.rng.Float64();
                rfg.i += 1;
                return x, true;
        } else {
                return 0.0, false;
        }
}

In [ ]:
extern crate rand;
use rand::Rng;

struct RandomFloatGenerator {
    a : f64,
    b : f64,
    n : i32,
    i : i32,
    rng : rand::rngs::ThreadRng
}

fn mk_random_float_generator(a : f64, b : f64, n : i32)
                           -> RandomFloatGenerator {
    let rng = rand::thread_rng();
    let i = 0;
    RandomFloatGenerator{a, b, n, i, rng}
}

impl RandomFloatGenerator {
    fn next(&mut self) -> Option<f64> {
        if self.i < self.n {
            let x = self.a + (self.b - self.a) * self.rng.gen::<f64>();
            self.i += 1;
            return Some(x)
        } else {
            return None
        }
    }
}

# 5. A minimizer for float -> float functions
## <font color="green"> Problem 3 :  A minimizer for float -> float functions</font>
Write a function `minimize`, which
 * takes $f$ (a function that takes a float and returns a float) and a `float_random_generator`
 * finds $x$ that minimizes $f(x)$ among those generated by the `float_random_generator`

 * In a pseudo code:
```
repeat {
  x = gen.next()
  if (no more values) break
  y = f(x)
}
return (x, f(x)) that attained the minimum value
```

* You have to handle a corner case in which the above loop iterates zero times, in which case you somehow need to indicate the returned values are meaningless
* The most natural way to do this again depends on the language


In [ ]:
BEGIN SOLUTION
END SOLUTION


In [ ]:
let take_min min_xy x y =
  match min_xy with
    None -> Some((x, y))
  | Some((min_x, min_y)) ->
     if y < min_y then
       Some((x, y))
     else
       min_xy
;;
let minimize f rfg =
  let rec loop min_xy =
    match rfg#next with
      None -> min_xy
    | Some(x) ->
       let y = f x in
       let min_xy = take_min min_xy x y in
       loop min_xy
  in loop None
;;

In [ ]:
function minimize(f, rfg)
    min_x = nothing
    min_y = nothing
    while true
        x = next(rfg)
        if x == nothing
            return (min_x, min_y)
        end
        y = f(x)
        if min_x == nothing || y < min_y
            min_x, min_y = x, y
        end
    end
end

In [ ]:
func minimize (f func (float64) float64, rfg * RandomFloatGenerator) (float64, float64, bool) {
        var min_x float64
        var min_y float64
        i := 0
        for {
                x, some := rfg.next()
                if ! some { break }
                y := f(x)
                if i == 0 || y < min_y {
                        min_y = y
                        min_x = x
                }
                i += 1
        }
        return min_x, min_y, i > 0
}

In [ ]:
fn take_min(min_xy : Option::<(f64, f64)>, x : f64, y : f64)
            -> Option::<(f64, f64)> {
    match min_xy {
        None => Some((x, y)),
        Some((_, min_y)) =>
            if y < min_y {
                Some((x, y))
            } else {
                min_xy
            }
    }
}

fn minimize(f : fn (f64) -> f64, rfg : &mut RandomFloatGenerator)
            -> Option::<(f64, f64)> {
    let mut min_xy : Option::<(f64, f64)> = None;
    loop {
        match rfg.next() {
            None => break,
            Some(x) => {
                let y = f(x);
                min_xy = take_min(min_xy, x, y);
            }
        }
    }
    min_xy
}

## <font color="green"> Problem 4 :  Apply float -> float minimizer</font>
Apply `minimize` to minimize function $f(x) = x(x-1)(x-2)$ in the interval $[0, 2]$


In [ ]:
BEGIN SOLUTION
END SOLUTION


In [ ]:
minimize (fun x -> x *. (x -. 1.0) *. (x -. 2.0)) (new random_float_generator 0.0 2.0 10000)
;;

In [ ]:
minimize(x -> x*(x-1)*(x-2), random_float_generator(0.0, 2.0, 10000))

In [ ]:
import "fmt"
func main() {
        f := func (x float64) float64 {
                return x * (x - 1.0) * (x - 2.0)
        }
        rfg := mk_random_float_generator(0.0, 2.0, 10000)
        x, y, some := minimize(f, rfg)
        if some {
                fmt.Printf("%f %f\n", x, y);
        } else {
                fmt.Printf("\n");
        }
}
main()

In [ ]:
fn main() {
    fn f(x : f64) -> f64 { x * (x - 1.0) * (x - 2.0) }
    let mut rfg = mk_random_float_generator(0.0, 2.0, 1000);
    match minimize(f, &mut rfg) {
        Some((x, y)) => println!("{} {}", x, y),
        None => println!("")
    }
}
main();

# 6. A first step to make it generic
* You will notice that the underlying principles or the algorithm dictated by the `minimize` function defined above generically works for domains other than floating point numbers; all it is doing is to draw an element from a domain and apply the function to it, with nothing specific to a floating point number
* It is therefore quite natural to desire it be generic in our programs too
* Remarkably, it is automatically happening in some languages (which one, do you think?)
* In other languages, the objective function $f$ explicitly takes a floating point number and your domain parameter is `float_random_generator`, both of which prevent us from applying the same function to any other domain

* Let's first fix the latter, that is, its domain parameter hard-coded to `float_random_generator`

## <font color="green"> Problem 5 :  A somewhat generic float -> float minimizer</font>
* If necessary in your language, make an interface or a similar mechanism, called `float_generator`, that represents any type that has `next()` method that returns a floating point number
* Clearly, `float_random_generator` implements `float_generator`; make it explicit if required in your language
* Fix the `minimize` function so as to takes any type that implements the above interface

In [ ]:
BEGIN SOLUTION
END SOLUTION


In [ ]:
(* same as above (Problem 3) *)

In [ ]:
# same as above (Problem 3)

In [ ]:
type FloatGenerator interface {
    next() (float64, bool)
}

func minimize (f func (float64) float64, rfg FloatGenerator) (float64, float64, bool) {
    var min_x float64
    var min_y float64
    i := 0
    for {
        x, some := rfg.next()
        if ! some { break }
        y := f(x)
        if i == 0 || y < min_y {
            min_y = y
            min_x = x
        }
        i += 1
    }
    return min_x, min_y, i > 0
}

In [ ]:
trait FloatGenerator {
    fn next(&mut self) -> Option<f64>;
}

impl FloatGenerator for RandomFloatGenerator {
    fn next(&mut self) -> Option<f64> {
        if self.i < self.n {
            let x = self.a + (self.b - self.a) * self.rng.gen::<f64>();
            self.i += 1;
            return Some(x)
        } else {
            return None
        }
    }
}

fn take_min(min_xy : Option::<(f64, f64)>, x : f64, y : f64)
            -> Option::<(f64, f64)> {
    match min_xy {
        None => Some((x, y)),
        Some((_, min_y)) =>
            if y < min_y {
                Some((x, y))
            } else {
                min_xy
            }
    }
}

fn minimize(f : fn (f64) -> f64, rfg : &mut dyn FloatGenerator)
            -> Option::<(f64, f64)> {
    let mut min_xy : Option::<(f64, f64)> = None;
    loop {
        match rfg.next() {
            None => break,
            Some(x) => {
                let y = f(x);
                min_xy = take_min(min_xy, x, y);
            }
        }
    }
    min_xy
}

## <font color="green"> Problem 6 :  Apply a somewhat generic float -> float minimizer</font>
Check the new `minimize` works for the same function $f(x) = x(x-1)(x-2)$ and the `float_random_generator`

In [ ]:
BEGIN SOLUTION
END SOLUTION


In [ ]:
(* same as above (Problem 4) *)

In [ ]:
# same as above (Problem 4)

In [ ]:
func main() {
    f := func (x float64) float64 {
        return x * (x - 1.0) * (x - 2.0)
    }
    rfg := mk_random_float_generator(0.0, 2.0, 10000)
    x, y, some := minimize(f, rfg)
    if some {
        fmt.Printf("%f %f\n", x, y);
    } else {
        fmt.Printf("\n");
    }
}
main()

In [ ]:
fn main() {
    fn f(x : f64) -> f64 { x * (x - 1.0) * (x - 2.0) }
    let mut rfg = mk_random_float_generator(0.0, 2.0, 1000);
    match minimize(f, &mut rfg) {
        Some((x, y)) => println!("{} {}", x, y),
        None => println!("")
    }
}
main();

# 7. Parametric types and functions
* The problems below require parametric types
* Write answers as standalone programs rather than in Jupyter, primarily because Go in Jupyter environment is not new enough (< 1.18) to support generic types and functions

# 8. A warm up for a generic type
## <font color="green"> Problem 7 :  Define a trivial generic type and a function</font>
* Master the syntax of parameterized data type, creating an instance of such types and parameterized functions
* As a practice, define a parameterized data type, called `triv`, that takes any value $x$ of any type $T$, and a function that takes a value of `triv` type and returns its value.
* That is, write a program that does something like the following in your language
```
s = triv(3)
x = triv_val(s) # should return 3
t = triv("hello")
y = triv_val(t) # should return "hello"
print(x)        # should print 3
print(y)        # should print hello
```

* The exact syntax and case conventions may differ depending on the language you chose
* <font color="red">Read the following document before working on the following problems</font>

 |     |                |                                         |
 |-----|----------------|-----------------------------------------|
 |Go   |generator[$T$]  |[Tutorial: Getting started with generics](https://go.dev/doc/tutorial/generics)|
 |Julia|generator{$T$}  |[Parametric Types](https://docs.julialang.org/en/v1/manual/types/#Parametric-Types)|
 |Julia|generator{$T$}  |[Parametric Methods](https://docs.julialang.org/en/v1/manual/methods/#Parametric-Methods)|
 |Rust |generator<$T$>  |[Generic Data Types](https://doc.rust-lang.org/book/ch10-01-syntax.html#generic-data-types)|
 |OCaml|$'a$ generator  |[Records and variants](https://ocaml.org/manual/coreexamples.html#s:tut-recvariants)|

* write a standalone program that defines the above data type (`triv`) and function (`triv_val`) and prints the value of `x` and `y`
* it must be written under `~/notebooks/pl05_generic_fun_type/p7/<lang>` directory, where &lt;lang&gt; is one of ml, jl, go and rs
* always create `triv` folder under the above directory, either manually or by creating a project with the respective build system
* as a result, the source file name must be

  * Go    : `~/notebooks/pl05_generic_fun_type/p7/go/triv/triv.go`
  * Julia : `~/notebooks/pl05_generic_fun_type/p7/jl/triv/triv.jl`
  * OCaml : `~/notebooks/pl05_generic_fun_type/p7/ml/triv/bin/main.ml`
  * Rust  : `~/notebooks/pl05_generic_fun_type/p7/rs/triv/src/main.rs`
* after writing the program, compile and run it with one of the following

* Go

In [ ]:
cd ~/notebooks/pl05_generic_fun_type/p7/go/triv
go build
./triv

* Julia

In [ ]:
cd ~/notebooks/pl05_generic_fun_type/p7/jl/triv
julia triv.jl

* OCaml 

In [ ]:
eval $(opam env)
cd ~/notebooks/pl05_generic_fun_type/p7/ml/triv
dune build
./_build/default/bin/main.exe

* Rust

In [ ]:
cd ~/notebooks/pl05_generic_fun_type/p7/rs/triv
cargo build
./target/debug/triv

* The output should be
```
3
hello
```

# 9. Generic (parameterized) functions and types
* Now we are ready to make `minimize` truly generic, in the sense that it can now minimize a function whose input domain is not a floating point number
* To this end, you need to _parameterize_ `float_generator` and `minimize`
* Let's call it `generator`; it can take a type parameter $T$ and its `next()` method now generates a value of $T$

## <font color="green"> Problem 8 :  A generic T -> float minimizer</font>
* Parameterize the generator type, as well as the type of incoming parameters of `minimize`, so that it can minimize functions taking values other than a floating point number (e.g., an integer or a pair of floats)
* Then write a generator `ellipse_generator`, which takes an ellipse's center $(x_0, y_0)$ and horizontal and vertical radius $a$ and $b$ upon creation and whose `next()` method returns a pair of floats inside the ellipse. That is, `next()` should return a random point $(x, y)$ satisfying
$$\frac{(x-x_0)^2}{a^2} + \frac{(y-y_0)^2}{b^2} \leq 1$$
* Minimize function $f(x, y) = x^2 + y^2$ in the ellipse whose center $= (3, 3)$, $a = 2$ and $b = 1$; and print $x$ and $y$ that minimize $f$, as well as $f(x, y)$.

* Implement it as a standalone program
* It must be written under `~/notebooks/pl05_generic_fun_type/p8/<lang>` directory, where &lt;lang&gt; is one of ml, jl, go and rs
* always create `minimize` folder under the above directory, either manually or by creating a project with the respective build system
* as a result, the source file name must be

  * Go    : `~/notebooks/pl05_generic_fun_type/p7/go/minimize/minimize.go`
  * Julia : `~/notebooks/pl05_generic_fun_type/p7/jl/minimize/minimize.jl`
  * OCaml : `~/notebooks/pl05_generic_fun_type/p7/ml/minimize/bin/main.ml`
  * Rust  : `~/notebooks/pl05_generic_fun_type/p7/rs/minimize/src/main.rs`
* after writing the program, compile and run it with one of the following

* Go

In [ ]:
cd ~/notebooks/pl05_generic_fun_type/p7/go/minimize
go build
./minimize

* Julia

In [ ]:
cd ~/notebooks/pl05_generic_fun_type/p7/jl/minimize
julia minimize.jl

* OCaml 

In [ ]:
eval $(opam env)
cd ~/notebooks/pl05_generic_fun_type/p7/ml/minimize
dune build
./_build/default/bin/main.exe

* Rust

In [ ]:
cd ~/notebooks/pl05_generic_fun_type/p7/rs/minimize
cargo build
./target/debug/minimize